# Tree Experiment

This notebook to build some javascript canvas infrastructures for
exploring tree geneartion algoritms. 


## Fractal Tree

In [8]:
%%html
<div id="container-tree">
    <div id="control-tree" style="position:absolute; right:0; top: 0"></div>
    <canvas id="canvas-tree" width="600" height="500"></canvas>
</div>

In [9]:
%%javascript

require.config({
    paths: {
        'dat.gui': 'https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.6/dat.gui.min'
    }
});

var $canvas = $('#canvas-tree');
var $controlContainer = $('#control-tree')
var ctx = $canvas[0].getContext('2d');

function addGui(gui, parent) {
  if(!parent) {
    parent = DAT.GUI.autoPlaceContainer;
  }
  parent.appendChild(gui);
}

(function(element) {
    require(['dat.gui'], function(dat) {   
        
        function redraw() {
            ctx.clearRect(0, 0, $canvas.width(), $canvas.height());
            drawTree(300, 500, -90, param.depth);
        }
        
        var Param = function() {
          this.depth = 9;
          this.angle = 20;
          this.applyWidth = true;
          this.random = function() {redraw()};
        };
        
        var param = new Param();
        var gui = new dat.GUI({
            autoPlace: false
        });
        $controlContainer.empty()
        $controlContainer.append(gui.domElement);

        gui.add(param, 'depth', 1, 20).onChange(() => redraw());
        gui.add(param, 'angle', 0, 90).onChange(() => redraw());
        gui.add(param, 'applyWidth').onChange(() => redraw());
        gui.add(param, 'random');

        ctx.fillStyle = '#C0C0C0';

        var deg_to_rad = Math.PI / 180.0;
        var depth = param.depth;

        function drawLine(x1, y1, x2, y2, brightness){
          ctx.beginPath();
          ctx.lineWidth = brightness * 0.7;
          ctx.moveTo(x1, y1);
          ctx.lineTo(x2, y2);
          ctx.closePath();
          ctx.stroke();
        }

        function drawTree(x1, y1, angle, depth){
          if (depth <= 0) return;

          var x2 = x1 + (Math.cos(angle * deg_to_rad) * (depth) * 10.0);
          var y2 = y1 + (Math.sin(angle * deg_to_rad) * (depth) * 10.0);
          drawLine(x1, y1, x2, y2, param.applyWidth ? depth : 1);
          if (Math.random() >= 0.45); drawTree(x2, y2, angle - param.angle, depth - 1);
          if (Math.random() >= 0.45) drawTree(x2, y2, angle + param.angle, depth - 1);
        }

        redraw();
    })
})(element);

<IPython.core.display.Javascript object>

Of course the fractal tree algorithm itself produces very PREMITIVE trees. I have plans to further tuning the algorithm to either make trees more realistic and find any related characteristics for an actual data-viz work driven by real data.

This is also served as a proof-of-concept for leveraging modern frontend technologies for viz experiments (in this case, canvas + javascript + [dat.gui](http://workshop.chromeexperiments.com/examples/gui/#1--Basic-Usage)). 
I feel excited and promising about this path since such workflow really powerful and opens flexibily, enables fast-prototyping and presents _Ling's viz dictionary idea_ in a good way - think of each interactable value can be driven by real data.

## L-System Generation

In [10]:
%%javascript

// ----------------------------------------------------
// Treer2d by Arnaud Couturier, improve by Grayson Wen
var trees2d = {};
trees2d.tree = function (canvas2d) {
    this.canvas2d = canvas2d;
    this.ctx2d = canvas2d.getContext("2d");
    this.string = "lL[+L][-L]";
    this.branchTexture;
    this.treePosY = 50;
    this.iterations = 10;
    this.angleMean = 20 / 180 * Math.PI;
    this.length = 30;
    this.lengthReduction = 0.8;
    this.thickness = 2;
    this.thicknessReduction = 0.65;
    this.rules = {
        L: {developsInto: ["l", "+lL", "-lL", "L[+LL][-LL]l", "[+L][-L]", "L[+lLL]", "L[-lLL]"]},
        l: {developsInto: ["l"]},
        "[": {developsInto: ["["]},
        "]": {developsInto: ["]"]},
        "+": {developsInto: ["+"]},
        "-": {developsInto: ["-"]}
    };
    this.leafTextures = [];
    this.leafScaleVariation = 0.5;
    this.leafMinDepth = 4;
    this.leafProba = 0.5;
    this.leafScale = 1;
    this.leafTotalPerBranch = 1;
    this.leafProbaLighterMult = 0.5
    this.shadowProba = 0.2;
    this.shadowAlpha = 0.025;
    this.shadowRadius = 40
};
trees2d.tree.prototype.addLeafTexture = function (a, b, c) {
    this.leafTextures.push({img: a, targetWidthInPixel: b, relativeProba: c})
};
trees2d.tree.prototype.generateLString = function () {
    // L-string expansion
    var _str = this.string;
    var lstring;

    for (var m = 0; m < this.iterations; m++) {
        lstring = "";
        for (var s = 0; s < _str.length; s++) {
            var _ch = _str[s];
            var _developsInto = this.rules[_ch].developsInto;
            if (_developsInto.length <= 0) {
                continue
            }
            // Randomly pick one of the devInto
            var a = _developsInto[parseInt(Math.random() * _developsInto.length)];

            lstring += a
        }
        _str = lstring
    }
    return lstring
};
trees2d.tree.prototype.removeLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            this.leafTextures.splice(b, 1);
            break
        }
    }
};
trees2d.tree.prototype.hasLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            return true
        }
    }
    return false
};
trees2d.tree.prototype.totalLeafTextures = function () {
    return this.leafTextures.length
};
trees2d.tree.prototype.draw = function (lstring) {
    for (var s = 0; s < this.leafTextures.length; s++) {
        var l = this.leafTextures[s];
        if (!l.img.complete) {
            return l.img.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (l.img.naturalWidth === 0 || l.img.naturalHeight === 0) {
            return l.img.src.substring(0, 100) + " is not a valid image"
        }
    }
    if (this.branchTexture) {
        if (!this.branchTexture.complete) {
            return this.branchTexture.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (this.branchTexture.naturalWidth === 0 || this.branchTexture.naturalHeight === 0) {
            return this.branchTexture.src.substring(0, 100) + " is not a valid image"
        }
    }
    var ctx = this.ctx2d;
    var err;
    ctx.setTransform(1, 0, 0, 1, 0, 0);
    ctx.save();
    try {

        var w = 0.1;
        var f = 1 / this.iterations;
        var strokeStyle = this.branchTexture ? this.ctx2d.createPattern(this.branchTexture, "repeat") : 'black';
        ctx.strokeStyle = strokeStyle;
        ctx.lineWidth = this.thickness;
        ctx.lineCap = "round";
        var p = 1;
        var h = 0;
        ctx.clearRect(0, 0, this.canvas2d.width, this.canvas2d.height);
        ctx.translate(this.canvas2d.width / 2, this.canvas2d.height - this.treePosY);
        for (var s = 0; s < lstring.length; s++) {
            switch (lstring[s]) {
                case "l":
                    ctx.strokeStyle = strokeStyle;
                    ctx.beginPath();
                    ctx.moveTo(0, 0);
                    ctx.quadraticCurveTo(0, - this.length / 2, 0, -this.length);
                    ctx.stroke();

                    if (Math.random() <= this.shadowProba) {
                        // Draw dark shadow
                        ctx.save();
                        ctx.globalCompositeOperation = "source-atop";
                        ctx.globalAlpha = this.shadowAlpha;
                        ctx.beginPath();
                        ctx.arc(0, 0, this.shadowRadius, 0, Math.PI * 2, false);
                        ctx.fill();
                        ctx.restore()
                    }
                    h++;
                    ctx.translate(0, -this.length);

                    // Draw leaf here
                    if (this.leafTextures.length > 0 && p >= this.leafMinDepth && Math.random() <= this.leafProba) {
                        for (var q = 0; q < this.leafTotalPerBranch; q++) {
                            ctx.save();
                            if (Math.random() <= (p / this.iterations * this.leafProbaLighterMult)) {
                                ctx.globalCompositeOperation = "lighter"
                            }
                            // Randomly pick a load leaf texture
                            var _leafTexture = this.leafTextures[parseInt(Math.random() * this.leafTextures.length)];
                            var u = _leafTexture.targetWidthInPixel / _leafTexture.img.naturalWidth * this.leafScale;

                            ctx.scale(
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation),
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation)
                            );
                            ctx.rotate(Math.random() * Math.PI * 2);
                            ctx.drawImage(_leafTexture.img, 0, 0);
                            ctx.restore()
                        }
                    }
                    break;
                case "+":
                    // Rotate left with variation
                    ctx.rotate(this.angleMean);
                    break;
                case "-":
                    // Rotate right with variation
                    ctx.rotate(-this.angleMean);
                    break;
                case "[":
                    ctx.save();
                    this.length *= this.lengthReduction;
                    p++;
                    w += f;
                    ctx.lineWidth *= this.thicknessReduction;
                    break;
                case "]":
                    this.length *= 1 / this.lengthReduction;
                    p--;
                    w -= f;
                    ctx.restore();
                    break;
            }
        }
    } catch (e) {
        err = e
    }
    ctx.restore();
    return err
};

// ----------------------------------------------------

require.config({
    paths: {
        'dat.gui': 'https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.6/dat.gui.min'
    }
});

var treeGenParamValues = {
    tp_iterations: {min: 1, max: 16, step: 1}
};
var treeParamValues = {
    tp_length: {min: 1, max: 200, step: 1},
    tp_angleMean: {min: 0, max: Math.PI / 4, step: Math.PI / 4 / 180},
    tp_thickness: {min: 1, max: 150, step: .1},
    tp_lengthReduction: {min: .5, max: 1, step: .01},
    tp_thicknessReduction: {min: .5, max: 1, step: .01},
};

(function(element) {
    var $canvas = $('<canvas width="800" height="500"></canvas>');
    var $controlContainer = $('<div id="control-tree" style="position:absolute; right:0; top: 0"></div>');
    var $container = $('<div></div>');
    $container
        .append($controlContainer)
        .append($canvas);
    element.append($container);
    
    var tree = new trees2d.tree($canvas[0]);
    var data = {
        lstr: '',
        error: ''
    }
    data.lstr = tree.generateLString();
    data.error = tree.draw(data.lstr);
    
    require(['dat.gui'], (dat) => {   
        
        var gui = new dat.GUI({autoPlace: false});
        $controlContainer.empty()
        $controlContainer.append(gui.domElement);
        
        var methods = {
            generate: () => { data.lstr = tree.generateLString(); data.error = tree.draw(data.lstr) }
        }
        
        for (var _i in treeGenParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeGenParamValues[_i].min, treeGenParamValues[_i].max)
                .onChange(() => methods.generate() );
        }
        
        gui.add(methods, 'generate');
        
        for (var _i in treeParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeParamValues[_i].min, treeParamValues[_i].max)
                .onChange(() => data.error = tree.draw(data.lstr));
        }
        
    })
})(element);


<IPython.core.display.Javascript object>

## Enable some variations

In [11]:
%%javascript

// ----------------------------------------------------
// Treer2d by Arnaud Couturier, improve by Grayson Wen
var trees2d = {};
trees2d.normalVariate = function (mean, variance) {
    return mean + (Math.random() * 2 + Math.random() * 2 + Math.random() * 2 - 3) / 3 * variance / 2
};
trees2d.tree = function (canvas2d) {
    this.canvas2d = canvas2d;
    this.ctx2d = canvas2d.getContext("2d");
    this.string = "lL[+L][-L]";
    this.branchTexture;
    this.treePosY = 50;
    this.iterations = 10;
    this.angleMean = 20 / 180 * Math.PI;
    this.angleVariation = 10 / 180 * Math.PI;
    this.length = 30;
    this.lengthReduction = 0.8;
    this.thickness = 6;
    this.thicknessReduction = 0.65;
    this.rules = {
        L: {developsInto: ["l", "+lL", "-lL", "L[+LL][-LL]l", "[+L][-L]", "L[+lLL]", "L[-lLL]"]},
        l: {developsInto: ["l"]},
        "[": {developsInto: ["["]},
        "]": {developsInto: ["]"]},
        "+": {developsInto: ["+"]},
        "-": {developsInto: ["-"]}
    };
    this.leafTextures = [];
    this.leafScaleVariation = 0.5;
    this.leafMinDepth = 4;
    this.leafProba = 0.5;
    this.leafScale = 1;
    this.leafTotalPerBranch = 1;
    this.leafProbaLighterMult = 0.5;
    this.curveXVariation = 10;
    this.curveYVariation = 5;
    this.shadowProba = 0.2;
    this.shadowAlpha = 0.025;
    this.shadowRadius = 40
};
trees2d.tree.prototype.addLeafTexture = function (a, b, c) {
    this.leafTextures.push({img: a, targetWidthInPixel: b, relativeProba: c})
};
trees2d.tree.prototype.generateLString = function () {
    // L-string expansion
    var _str = this.string;
    var lstring;

    for (var m = 0; m < this.iterations; m++) {
        lstring = "";
        for (var s = 0; s < _str.length; s++) {
            var _ch = _str[s];
            var _developsInto = this.rules[_ch].developsInto;
            if (_developsInto.length <= 0) {
                continue
            }
            // Randomly pick one of the devInto
            var a = _developsInto[parseInt(Math.random() * _developsInto.length)];

            lstring += a
        }
        _str = lstring
    }
    return lstring
};
trees2d.tree.prototype.removeLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            this.leafTextures.splice(b, 1);
            break
        }
    }
};
trees2d.tree.prototype.hasLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            return true
        }
    }
    return false
};
trees2d.tree.prototype.totalLeafTextures = function () {
    return this.leafTextures.length
};
trees2d.tree.prototype.draw = function (lstring) {
    for (var s = 0; s < this.leafTextures.length; s++) {
        var l = this.leafTextures[s];
        if (!l.img.complete) {
            return l.img.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (l.img.naturalWidth === 0 || l.img.naturalHeight === 0) {
            return l.img.src.substring(0, 100) + " is not a valid image"
        }
    }
    if (this.branchTexture) {
        if (!this.branchTexture.complete) {
            return this.branchTexture.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (this.branchTexture.naturalWidth === 0 || this.branchTexture.naturalHeight === 0) {
            return this.branchTexture.src.substring(0, 100) + " is not a valid image"
        }
    }
    var ctx = this.ctx2d;
    var err;
    ctx.setTransform(1, 0, 0, 1, 0, 0);
    ctx.save();
    try {

        var w = 0.1;
        var f = 1 / this.iterations;
        var strokeStyle = this.branchTexture ? this.ctx2d.createPattern(this.branchTexture, "repeat") : 'black';
        ctx.strokeStyle = strokeStyle;
        ctx.lineWidth = this.thickness;
        ctx.lineCap = "round";
        var p = 1;
        var h = 0;
        ctx.clearRect(0, 0, this.canvas2d.width, this.canvas2d.height);
        ctx.translate(this.canvas2d.width / 2, this.canvas2d.height - this.treePosY);
        for (var s = 0; s < lstring.length; s++) {
            switch (lstring[s]) {
                case "l":
                    ctx.strokeStyle = strokeStyle;
                    ctx.beginPath();
                    ctx.moveTo(0, 0);
                    ctx.quadraticCurveTo(trees2d.normalVariate(0, this.curveXVariation), trees2d.normalVariate(0, this.curveYVariation) - this.length / 2, 0, -this.length);
                    ctx.stroke();

                    if (Math.random() <= this.shadowProba) {
                        // Draw dark shadow
                        ctx.save();
                        ctx.globalCompositeOperation = "source-atop";
                        ctx.globalAlpha = this.shadowAlpha;
                        ctx.beginPath();
                        ctx.arc(0, 0, this.shadowRadius, 0, Math.PI * 2, false);
                        ctx.fill();
                        ctx.restore()
                    }
                    h++;
                    ctx.translate(0, -this.length);

                    // Draw leaf here
                    if (this.leafTextures.length > 0 && p >= this.leafMinDepth && Math.random() <= this.leafProba) {
                        for (var q = 0; q < this.leafTotalPerBranch; q++) {
                            ctx.save();
                            if (Math.random() <= (p / this.iterations * this.leafProbaLighterMult)) {
                                ctx.globalCompositeOperation = "lighter"
                            }
                            // Randomly pick a load leaf texture
                            var _leafTexture = this.leafTextures[parseInt(Math.random() * this.leafTextures.length)];
                            var u = _leafTexture.targetWidthInPixel / _leafTexture.img.naturalWidth * this.leafScale;

                            ctx.scale(
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation),
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation)
                            );
                            ctx.rotate(Math.random() * Math.PI * 2);
                            ctx.drawImage(_leafTexture.img, 0, 0);
                            ctx.restore()
                        }
                    }
                    break;
                case "+":
                    // Rotate left with variation
                    ctx.rotate(trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "-":
                    // Rotate right with variation
                    ctx.rotate(-trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "[":
                    ctx.save();
                    this.length *= this.lengthReduction;
                    p++;
                    w += f;
                    ctx.lineWidth *= this.thicknessReduction;
                    break;
                case "]":
                    this.length *= 1 / this.lengthReduction;
                    p--;
                    w -= f;
                    ctx.restore();
                    break;
            }
        }
    } catch (e) {
        err = e
    }
    ctx.restore();
    return err
};

// ----------------------------------------------------

require.config({
    paths: {
        'dat.gui': 'https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.6/dat.gui.min'
    }
});

var treeGenParamValues = {
    tp_iterations: {min: 1, max: 16, step: 1}
};
var treeParamValues = {
    tp_length: {min: 1, max: 200, step: 1},
    tp_angleVariation:{min:0, max:Math.PI/2, step:Math.PI/2/180},
    tp_angleMean: {min: 0, max: Math.PI / 4, step: Math.PI / 4 / 180},
    tp_thickness: {min: 1, max: 150, step: .1},
    tp_lengthReduction: {min: .5, max: 1, step: .01},
    tp_thicknessReduction: {min: .5, max: 1, step: .01},
    tp_curveXVariation: {min: 0, max: 100, step: .1},
    tp_curveYVariation: {min: 0, max: 100, step: .1},
};

(function(element) {
    var $canvas = $('<canvas width="600" height="500"></canvas>');
    var $controlContainer = $('<div id="control-tree" style="position:absolute; right:0; top: 0"></div>');
    var $container = $('<div></div>');
    $container
        .append($controlContainer)
        .append($canvas);
    element.append($container);
    
    var tree = new trees2d.tree($canvas[0]);
    var data = {
        lstr: '',
        error: ''
    }
    data.lstr = tree.generateLString();
    data.error = tree.draw(data.lstr);
    
    require(['dat.gui'], (dat) => {
        
        var gui = new dat.GUI({autoPlace: false});
        $controlContainer.empty()
        $controlContainer.append(gui.domElement);
        
        var methods = {
            generate: () => { data.lstr = tree.generateLString(); data.error = tree.draw(data.lstr) }
        }
        
        for (var _i in treeGenParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeGenParamValues[_i].min, treeGenParamValues[_i].max)
                .onChange(() => methods.generate() );
        }
        
        gui.add(methods, 'generate');
        
        for (var _i in treeParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeParamValues[_i].min, treeParamValues[_i].max)
                .onChange(() => data.error = tree.draw(data.lstr));
        }
        
    })
})(element);


<IPython.core.display.Javascript object>

## Texture Experiments

In [12]:
%%HTML
<img width="200px" src="https://images.unsplash.com/photo-1570454618940-165d5cc494f3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=634&q=80">

In [13]:
%%javascript

// ----------------------------------------------------
// Treer2d by Arnaud Couturier, improve by Grayson Wen
var trees2d = {};
trees2d.normalVariate = function (mean, variance) {
    return mean + (Math.random() * 2 + Math.random() * 2 + Math.random() * 2 - 3) / 3 * variance / 2
};
trees2d.tree = function (canvas2d) {
    this.canvas2d = canvas2d;
    this.ctx2d = canvas2d.getContext("2d");
    this.string = "lL[+L][-L]";
    this.branchTexture;
    this.treePosY = 50;
    this.iterations = 10;
    this.angleMean = 20 / 180 * Math.PI;
    this.angleVariation = 10 / 180 * Math.PI;
    this.length = 30;
    this.lengthReduction = 0.8;
    this.thickness = 15;
    this.thicknessReduction = 0.65;
    this.rules = {
        L: {developsInto: ["l", "+lL", "-lL", "L[+LL][-LL]l", "[+L][-L]", "L[+lLL]", "L[-lLL]"]},
        l: {developsInto: ["l"]},
        "[": {developsInto: ["["]},
        "]": {developsInto: ["]"]},
        "+": {developsInto: ["+"]},
        "-": {developsInto: ["-"]}
    };
    this.leafTextures = [];
    this.leafScaleVariation = 0.5;
    this.leafMinDepth = 4;
    this.leafProba = 0.5;
    this.leafScale = 1;
    this.leafTotalPerBranch = 1;
    this.leafProbaLighterMult = 0.5;
    this.curveXVariation = 10;
    this.curveYVariation = 5;
    this.shadowProba = 0.2;
    this.shadowAlpha = 0.025;
    this.shadowRadius = 40
};
trees2d.tree.prototype.addLeafTexture = function (a, b, c) {
    this.leafTextures.push({img: a, targetWidthInPixel: b, relativeProba: c})
};
trees2d.tree.prototype.generateLString = function () {
    // L-string expansion
    var _str = this.string;
    var lstring;

    for (var m = 0; m < this.iterations; m++) {
        lstring = "";
        for (var s = 0; s < _str.length; s++) {
            var _ch = _str[s];
            var _developsInto = this.rules[_ch].developsInto;
            if (_developsInto.length <= 0) {
                continue
            }
            // Randomly pick one of the devInto
            var a = _developsInto[parseInt(Math.random() * _developsInto.length)];

            lstring += a
        }
        _str = lstring
    }
    return lstring
};
trees2d.tree.prototype.removeLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            this.leafTextures.splice(b, 1);
            break
        }
    }
};
trees2d.tree.prototype.hasLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            return true
        }
    }
    return false
};
trees2d.tree.prototype.totalLeafTextures = function () {
    return this.leafTextures.length
};
trees2d.tree.prototype.draw = function (lstring) {
    for (var s = 0; s < this.leafTextures.length; s++) {
        var l = this.leafTextures[s];
        if (!l.img.complete) {
            return l.img.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (l.img.naturalWidth === 0 || l.img.naturalHeight === 0) {
            return l.img.src.substring(0, 100) + " is not a valid image"
        }
    }
    if (this.branchTexture) {
        if (!this.branchTexture.complete) {
            return this.branchTexture.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (this.branchTexture.naturalWidth === 0 || this.branchTexture.naturalHeight === 0) {
            return this.branchTexture.src.substring(0, 100) + " is not a valid image"
        }
    }
    var ctx = this.ctx2d;
    var err;
    ctx.setTransform(1, 0, 0, 1, 0, 0);
    ctx.save();
    try {

        var w = 0.1;
        var f = 1 / this.iterations;
        var strokeStyle = this.branchTexture ? this.ctx2d.createPattern(this.branchTexture, "repeat") : 'black';
        ctx.strokeStyle = strokeStyle;
        ctx.lineWidth = this.thickness;
        ctx.lineCap = "round";
        var p = 1;
        var h = 0;
        ctx.clearRect(0, 0, this.canvas2d.width, this.canvas2d.height);
        ctx.translate(this.canvas2d.width / 2, this.canvas2d.height - this.treePosY);
        for (var s = 0; s < lstring.length; s++) {
            switch (lstring[s]) {
                case "l":
                    ctx.strokeStyle = strokeStyle;
                    ctx.beginPath();
                    ctx.moveTo(0, 0);
                    ctx.quadraticCurveTo(trees2d.normalVariate(0, this.curveXVariation), trees2d.normalVariate(0, this.curveYVariation) - this.length / 2, 0, -this.length);
                    ctx.stroke();

                    if (Math.random() <= this.shadowProba) {
                        // Draw dark shadow
                        ctx.save();
                        ctx.globalCompositeOperation = "source-atop";
                        ctx.globalAlpha = this.shadowAlpha;
                        ctx.beginPath();
                        ctx.arc(0, 0, this.shadowRadius, 0, Math.PI * 2, false);
                        ctx.fill();
                        ctx.restore()
                    }
                    h++;
                    ctx.translate(0, -this.length);

                    // Draw leaf here
                    if (this.leafTextures.length > 0 && p >= this.leafMinDepth && Math.random() <= this.leafProba) {
                        for (var q = 0; q < this.leafTotalPerBranch; q++) {
                            ctx.save();
                            if (Math.random() <= (p / this.iterations * this.leafProbaLighterMult)) {
                                ctx.globalCompositeOperation = "lighter"
                            }
                            // Randomly pick a load leaf texture
                            var _leafTexture = this.leafTextures[parseInt(Math.random() * this.leafTextures.length)];
                            var u = _leafTexture.targetWidthInPixel / _leafTexture.img.naturalWidth * this.leafScale;

                            ctx.scale(
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation),
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation)
                            );
                            ctx.rotate(Math.random() * Math.PI * 2);
                            ctx.drawImage(_leafTexture.img, 0, 0);
                            ctx.restore()
                        }
                    }
                    break;
                case "+":
                    // Rotate left with variation
                    ctx.rotate(trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "-":
                    // Rotate right with variation
                    ctx.rotate(-trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "[":
                    ctx.save();
                    this.length *= this.lengthReduction;
                    p++;
                    w += f;
                    ctx.lineWidth *= this.thicknessReduction;
                    break;
                case "]":
                    this.length *= 1 / this.lengthReduction;
                    p--;
                    w -= f;
                    ctx.restore();
                    break;
            }
        }
    } catch (e) {
        err = e
    }
    ctx.restore();
    return err
};

// ----------------------------------------------------

require.config({
    paths: {
        'dat.gui': 'https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.6/dat.gui.min'
    }
});

var treeGenParamValues = {
    tp_iterations: {min: 1, max: 16, step: 1}
};
var treeParamValues = {
    tp_length: {min: 1, max: 200, step: 1},
    tp_shadowAlpha: {min: 0, max: .1, step: .001},
    tp_shadowProba: {min: 0, max: .3, step: .001},
    tp_shadowRadius: {min: 0, max: 200, step: .5},
    tp_angleVariation:{min:0, max:Math.PI/2, step:Math.PI/2/180},
    tp_angleMean: {min: 0, max: Math.PI / 4, step: Math.PI / 4 / 180},
    tp_thickness: {min: 1, max: 150, step: .1},
    tp_lengthReduction: {min: .5, max: 1, step: .01},
    tp_thicknessReduction: {min: .5, max: 1, step: .01},
    tp_curveXVariation: {min: 0, max: 100, step: .1},
    tp_curveYVariation: {min: 0, max: 100, step: .1},
};

(function(element) {
    var $canvas = $('<canvas width="600" height="500"></canvas>');
    var $controlContainer = $('<div id="control-tree" style="position:absolute; right:0; top: 0"></div>');
    var $container = $('<div></div>');
    var $branchTexture = $('<img width="40px" src="https://images.unsplash.com/photo-1570454618940-165d5cc494f3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=634&q=80">')
    $container
        .append($controlContainer)
        .append($canvas);
    element.append($branchTexture);
    element.append($container);
    
    var tree = new trees2d.tree($canvas[0]);
    tree.branchTexture = $branchTexture[0];
    var data = {
        lstr: '',
        error: ''
    }
    data.lstr = tree.generateLString();
    data.error = tree.draw(data.lstr);
    
    require(['dat.gui'], (dat) => {
        
        var gui = new dat.GUI({autoPlace: false});
        $controlContainer.empty()
        $controlContainer.append(gui.domElement);
        
        var methods = {
            generate: () => { data.lstr = tree.generateLString(); data.error = tree.draw(data.lstr) }
        }
        
        for (var _i in treeGenParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeGenParamValues[_i].min, treeGenParamValues[_i].max)
                .onChange(() => methods.generate() );
        }
        
        gui.add(methods, 'generate');
        
        for (var _i in treeParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeParamValues[_i].min, treeParamValues[_i].max)
                .onChange(() => data.error = tree.draw(data.lstr));
        }
        
    })
})(element);


<IPython.core.display.Javascript object>

In [14]:
%%javascript

// ----------------------------------------------------
// Treer2d by Arnaud Couturier, improve by Grayson Wen
var trees2d = {};
trees2d.normalVariate = function (mean, variance) {
    return mean + (Math.random() * 2 + Math.random() * 2 + Math.random() * 2 - 3) / 3 * variance / 2
};
trees2d.tree = function (canvas2d) {
    this.canvas2d = canvas2d;
    this.ctx2d = canvas2d.getContext("2d");
    this.string = "lL[+L][-L]";
    this.branchTexture;
    this.treePosY = 50;
    this.iterations = 10;
    this.angleMean = 20 / 180 * Math.PI;
    this.angleVariation = 10 / 180 * Math.PI;
    this.length = 30;
    this.lengthReduction = 0.8;
    this.thickness = 15;
    this.thicknessReduction = 0.65;
    this.rules = {
        L: {developsInto: ["l", "+lL", "-lL", "L[+LL][-LL]l", "[+L][-L]", "L[+lLL]", "L[-lLL]"]},
        l: {developsInto: ["l"]},
        "[": {developsInto: ["["]},
        "]": {developsInto: ["]"]},
        "+": {developsInto: ["+"]},
        "-": {developsInto: ["-"]}
    };
    this.leafTextures = [];
    this.leafScaleVariation = 0.5;
    this.leafMinDepth = 4;
    this.leafProba = 0.5;
    this.leafScale = 1;
    this.leafTotalPerBranch = 1;
    this.leafProbaLighterMult = 0.5;
    this.curveXVariation = 10;
    this.curveYVariation = 5;
    this.shadowProba = 0.2;
    this.shadowAlpha = 0.025;
    this.shadowRadius = 40
};
trees2d.tree.prototype.addLeafTexture = function (a, b, c) {
    this.leafTextures.push({img: a, targetWidthInPixel: b, relativeProba: c})
};
trees2d.tree.prototype.generateLString = function () {
    // L-string expansion
    var _str = this.string;
    var lstring;

    for (var m = 0; m < this.iterations; m++) {
        lstring = "";
        for (var s = 0; s < _str.length; s++) {
            var _ch = _str[s];
            var _developsInto = this.rules[_ch].developsInto;
            if (_developsInto.length <= 0) {
                continue
            }
            // Randomly pick one of the devInto
            var a = _developsInto[parseInt(Math.random() * _developsInto.length)];

            lstring += a
        }
        _str = lstring
    }
    return lstring
};
trees2d.tree.prototype.removeLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            this.leafTextures.splice(b, 1);
            break
        }
    }
};
trees2d.tree.prototype.hasLeafTexture = function (a) {
    for (var b = 0; b < this.leafTextures.length; b++) {
        var c = this.leafTextures[b];
        if (c.img == a) {
            return true
        }
    }
    return false
};
trees2d.tree.prototype.totalLeafTextures = function () {
    return this.leafTextures.length
};
trees2d.tree.prototype.draw = function (lstring) {
    for (var s = 0; s < this.leafTextures.length; s++) {
        var l = this.leafTextures[s];
        if (!l.img.complete) {
            return l.img.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (l.img.naturalWidth === 0 || l.img.naturalHeight === 0) {
            return l.img.src.substring(0, 100) + " is not a valid image"
        }
    }
    if (this.branchTexture) {
        if (!this.branchTexture.complete) {
            return this.branchTexture.src.substring(0, 100) + " is not completely loaded, wait until all textures are loaded"
        }
        if (this.branchTexture.naturalWidth === 0 || this.branchTexture.naturalHeight === 0) {
            return this.branchTexture.src.substring(0, 100) + " is not a valid image"
        }
    }
    var ctx = this.ctx2d;
    var err;
    ctx.setTransform(1, 0, 0, 1, 0, 0);
    ctx.save();
    try {

        var w = 0.1;
        var f = 1 / this.iterations;
        var strokeStyle = this.branchTexture ? this.ctx2d.createPattern(this.branchTexture, "repeat") : 'black';
        ctx.strokeStyle = strokeStyle;
        ctx.lineWidth = this.thickness;
        ctx.lineCap = "round";
        var p = 1;
        var h = 0;
        ctx.clearRect(0, 0, this.canvas2d.width, this.canvas2d.height);
        ctx.translate(this.canvas2d.width / 2, this.canvas2d.height - this.treePosY);
        for (var s = 0; s < lstring.length; s++) {
            switch (lstring[s]) {
                case "l":
                    ctx.strokeStyle = strokeStyle;
                    ctx.beginPath();
                    ctx.moveTo(0, 0);
                    ctx.quadraticCurveTo(trees2d.normalVariate(0, this.curveXVariation), trees2d.normalVariate(0, this.curveYVariation) - this.length / 2, 0, -this.length);
                    ctx.stroke();

                    if (Math.random() <= this.shadowProba) {
                        // Draw dark shadow
                        ctx.save();
                        ctx.globalCompositeOperation = "source-atop";
                        ctx.globalAlpha = this.shadowAlpha;
                        ctx.beginPath();
                        ctx.arc(0, 0, this.shadowRadius, 0, Math.PI * 2, false);
                        ctx.fill();
                        ctx.restore()
                    }
                    h++;
                    ctx.translate(0, -this.length);

                    // Draw leaf here
                    if (this.leafTextures.length > 0 && p >= this.leafMinDepth && Math.random() <= this.leafProba) {
                        for (var q = 0; q < this.leafTotalPerBranch; q++) {
                            ctx.save();
                            if (Math.random() <= (p / this.iterations * this.leafProbaLighterMult)) {
                                ctx.globalCompositeOperation = "lighter"
                            }
                            // Randomly pick a load leaf texture
                            var _leafTexture = this.leafTextures[parseInt(Math.random() * this.leafTextures.length)];
                            var u = _leafTexture.targetWidthInPixel / _leafTexture.img.naturalWidth * this.leafScale;

                            ctx.scale(
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation),
                                u * (1 + (Math.random() * 2 - 1) * this.leafScaleVariation)
                            );
                            ctx.rotate(Math.random() * Math.PI * 2);
                            ctx.drawImage(_leafTexture.img, 0, 0);
                            ctx.restore()
                        }
                    }
                    break;
                case "+":
                    // Rotate left with variation
                    ctx.rotate(trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "-":
                    // Rotate right with variation
                    ctx.rotate(-trees2d.normalVariate(this.angleMean, this.angleVariation));
                    break;
                case "[":
                    ctx.save();
                    this.length *= this.lengthReduction;
                    p++;
                    w += f;
                    ctx.lineWidth *= this.thicknessReduction;
                    break;
                case "]":
                    this.length *= 1 / this.lengthReduction;
                    p--;
                    w -= f;
                    ctx.restore();
                    break;
            }
        }
    } catch (e) {
        err = e
    }
    ctx.restore();
    return err
};

// ----------------------------------------------------

require.config({
    paths: {
        'dat.gui': 'https://cdnjs.cloudflare.com/ajax/libs/dat-gui/0.7.6/dat.gui.min'
    }
});

var treeGenParamValues = {
    tp_iterations: {min: 1, max: 16, step: 1}
};
var treeParamValues = {
    tp_length: {min: 1, max: 200, step: 1},
    tp_shadowAlpha: {min: 0, max: .1, step: .001},
    tp_shadowProba: {min: 0, max: .3, step: .001},
    tp_shadowRadius: {min: 0, max: 200, step: .5},
    tp_angleVariation:{min:0, max:Math.PI/2, step:Math.PI/2/180},
    tp_angleMean: {min: 0, max: Math.PI / 4, step: Math.PI / 4 / 180},
    tp_thickness: {min: 1, max: 150, step: .1},
    tp_lengthReduction: {min: .5, max: 1, step: .01},
    tp_thicknessReduction: {min: .5, max: 1, step: .01},
    tp_curveXVariation: {min: 0, max: 100, step: .1},
    tp_curveYVariation: {min: 0, max: 100, step: .1},
};

(function(element) {
    var $canvas = $('<canvas width="600" height="500"></canvas>');
    var $controlContainer = $('<div id="control-tree" style="position:absolute; right:0; top: 0"></div>');
    var $container = $('<div></div>');
    var $branchTexture = $('<img width="40px" src="https://image.freepik.com/free-photo/tree-trunk-texture-close-up_23-2148189568.jpg">')
    $container
        .append($controlContainer)
        .append($canvas);
    element.append($branchTexture);
    element.append($container);
    
    var tree = new trees2d.tree($canvas[0]);
    tree.branchTexture = $branchTexture[0];
    var data = {
        lstr: '',
        error: ''
    }
    data.lstr = tree.generateLString();
    data.error = tree.draw(data.lstr);
    
    require(['dat.gui'], (dat) => {
        
        var gui = new dat.GUI({autoPlace: false});
        $controlContainer.empty()
        $controlContainer.append(gui.domElement);
        
        var methods = {
            generate: () => { data.lstr = tree.generateLString(); data.error = tree.draw(data.lstr) }
        }
        
        for (var _i in treeGenParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeGenParamValues[_i].min, treeGenParamValues[_i].max)
                .onChange(() => methods.generate() );
        }
        
        gui.add(methods, 'generate');
        
        for (var _i in treeParamValues) {
            var p = _i.substring(3, _i.length);
            //console.log(tree, _i, p, treeParamValues);
            gui.add(tree, p, treeParamValues[_i].min, treeParamValues[_i].max)
                .onChange(() => data.error = tree.draw(data.lstr));
        }
        
    })
})(element);


<IPython.core.display.Javascript object>